## Data preparation
#### Pablo Gonzalez B.

### Variable Description

`GRCODE NAIC` company code (including insurer groups and single insurers)

`GRNAME NAIC` company name (including insurer groups and single insurers)

`AccidentYear` Accident year(1988 to 1997)

`DevelopmentYear` Development year (1988 to 1997)

`DevelopmentLag` Development year (AY-1987 + DY-1987 - 1)

`IncurLoss_` Incurred losses and allocated expenses reported at year end

`CumPaidLoss_` Cumulative paid losses and allocated expenses at year end

`BulkLoss_` Bulk and IBNR reserves on net losses and defense and cost containment expenses reported at year end

`PostedReserve97_` Posted reserves in year 1997 taken from the Underwriting and Investment Exhibit – Part 2A, including net losses unpaid and unpaid loss adjustment expenses

`EarnedPremDIR_` Premiums earned at incurral year - direct and assumed

`EarnedPremCeded_` Premiums earned at incurral year - ceded

`EarnedPremNet_` Premiums earned at incurral year - net

`Single` 1 indicates a single entity, 0 indicates a group insurer

### Import libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats

### Read data

In [ ]:
data = pd.read_csv('medmal_pos.csv')
data.shape

In [ ]:
data.head()

### Data preprocessing

En primera instancia, la variable `GRCODE` es una variable identificadora, por lo tanto no es una feature relevante para nuestro modelo, procedemos a eliminarla.

In [ ]:
data = data.drop(columns=['GRCODE'])

Anteriormente vimos en el EDA que hay una correlación fuerte entre las siguientes features:
- `CumPaidLoss_F2`, `IncurLoss_F2`
- `EarnedPremDIR_F2`, `IncurLoss_F2`
- `EarnedPremDIR_F2`, `EarnedPremNet_F2`

Las demás correlaciones son inducidas por estas anteriores, por lo tanto para evitar heterocedasticidad en nuestro modelo optaremos por eliminar las features agregadas como por ejemplo `CumPaidLoss_F2` y `EarnedPremDIR_F2`.

Después de esto, quedamos con las siguientes correlaciones entre features:

- `PostedReserve97_F2`, `IncurLoss_F2`
- `PostedReserve97_F2`, `EarnedPremNet_F2`

Si observamos la variable `PostedReserve97_F2`, esta variable es constante para distintos valores del tiempo (`AccidentYear` o `DevelopmentYear`), esto puede representar un obstáculo si decidimos modelar nuestro problema como un problema de forecasting de series de tiempo, por lo tanto la eliminamos.

In [ ]:
data = data.drop(columns=['CumPaidLoss_F2', 'EarnedPremDIR_F2', 'PostedReserve97_F2'])

Ahora graficamos nuevamente nuestra matriz de correlaciones.

In [ ]:
plt.rcParams['figure.figsize'] = (22, 7)
numeric_features = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
sns.heatmap(data[numeric_features].corr(method='spearman'), annot=True);

Solo nos queda una correlación por eliminar para evitar heterocedasticidad, aquella que se observa entre las features `EarnedPremNet_F2` y `IncurLoss_F2`. Si lo que deseamos es hacer un forecast de la pérdida, nuestras candidatas a variables objetivo serán `IncurLoss_F2` o `BulkLoss_F2`, por lo tanto no tiene sentido eliminar alguna de estas features. Por otro lado, tampoco eliminaremos la feature `EarnedPremNet_F2` porque si `IncurLoss_F2` llega a ser la variable objetivo, estaremos eliminando una feature que probablemente pueda explicar fuertemente la variación de `IncurLoss_F2`, por lo tanto dejaremos nuestro dataset así, y ahora procedemos a escalar y codificar nuestras features.

En nuestro EDA vimos que contamos con valores atípicos en algunas de nuestras features numéricas, por lo tanto procedemos a eliminar dichos valores, para eso utilizaremos el z-score de la distribución de cada feature y eliminaremos los valores que estén a cierta distancia del z-score.

In [ ]:
processed_data = data[(np.abs(stats.zscore(data['IncurLoss_F2'])) < 3) & \
                      (np.abs(stats.zscore(data['BulkLoss_F2'])) < 3) & \
                      (np.abs(stats.zscore(data['EarnedPremCeded_F2'])) < 3) & \
                      (np.abs(stats.zscore(data['EarnedPremNet_F2'])) < 3)].copy().reset_index(drop=True)

numeric_features = processed_data.select_dtypes(include=['int64', 'float64']).columns.tolist()
non_numeric_features = processed_data.select_dtypes(include=['object']).columns.tolist()

Posteriormente normalizamos nuestra variable textual para evitar textos duplicados por typos.

In [ ]:
from cucco import Cucco

cucco = Cucco()

normalizations = [
    'remove_extra_white_spaces',
    ('replace_punctuation', {'replacement': ''})
]

# normalizamos la variable categorica GRNAME
processed_data['GRNAME'] = processed_data['GRNAME'].apply(lambda x : cucco.normalize(x, normalizations))

In [ ]:
processed_data.head()

Después de haber normalizado la feature categorica de texto, aplicamos one-hot encoding a dicha feature categorica usando `pd.get_dummies()` y usando `StandardScaler()` de `sklearn` estandarizamos las features numéricas menos `AccidentYear`, `DevelopmentYear`, `DevelopmentLag` y `Single`, pues estandarizar estas features para nuestro modelo no tendría sentido, mucho menos las features de año.

In [ ]:
from sklearn.preprocessing import StandardScaler

standard_scaler = StandardScaler()

year_single_features = processed_data[['AccidentYear', 'DevelopmentYear', 'DevelopmentLag', 'Single']].copy()
standardized = pd.DataFrame(standard_scaler.fit_transform(processed_data[numeric_features].copy().drop(columns=['AccidentYear', 'DevelopmentYear', 'DevelopmentLag', 'Single'])),
                 columns=processed_data[numeric_features].copy().drop(columns=['AccidentYear', 'DevelopmentYear', 'DevelopmentLag', 'Single']).columns)
one_hot_encoded = pd.get_dummies(processed_data[non_numeric_features].copy(), dtype=bool).replace({True : 1, False : 0})

processed_data_scaled = pd.concat([year_single_features, standardized, one_hot_encoded], axis=1)

In [ ]:
processed_data_scaled.head()

Verifiquemos que las features quedaron estandarizadas calculando su media y su desviación estándar.

In [ ]:
standardized_features = ['IncurLoss_F2', 'BulkLoss_F2', 'EarnedPremCeded_F2', 'EarnedPremNet_F2']
processed_data_scaled[standardized_features].describe().T[['mean', 'std']].round()

En esta instancia hemos eliminado correlaciones entre nuestas features que puedan causar heterocedasticidad, imputamos valores atípicos que pueden darle problemas a nuestro modelo, codificamos nuestras features categoricas y estandarizamos nuestras features numericas, en esta instancia nuestros datos están listos para que un modelo sea ajustado.